In [1]:
import tensorflow as tf
import numpy as np

# 학습 데이터. x는 두 값이 주어지고, y는 0 또는 1로 분류.
x_train = [[1., 2.],
          [2., 3.],
          [3., 1.],
          [4., 3.],
          [5., 3.],
          [6., 2.]]
y_train = [[0.],
          [0.],
          [0.],
          [1.],
          [1.],
          [1.]]

# 테스트 데이터
x_test = [[5.,2.]]
y_test = [[1.]]

"""
학습 데이터들을 from_tensor_slices를 통해 담는다.  
6개의 slices로 만들고 각각의 slice는 1행2열과 1행 크기를 갖게 된다.
"""
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# Weight와 bias는 0으로 초기 설정.
W = tf.Variable(tf.zeros([2,1]), name='weight')
b = tf.Variable(tf.zeros([1]), name='bias')

In [2]:
# 로지스틱 회귀 가설 함수
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(-tf.matmul(features, W) + b))
    return hypothesis


# 로그 손실 함수
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [3]:
"""
정확도 측정.
0.5보다 크면 1, 0.5보다 작으면 0으로 predicted를 정하고
tf.equal()로 실제값과 예측값이 같은지 다른지 판명한 후에
tf.cast()로 True->1, False->0 변경 후
평균을 구해 accuaracy에 담은 후 리턴한다.
"""
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy


def grad(features, labels):
    with tf.GradientTape() as tape:
        hypothesis = logistic_regression(features)
        loss_value = loss_fn(hypothesis,labels)
    return tape.gradient(loss_value, [W,b])

In [4]:
EPOCHS = 1001

for step in range(EPOCHS):
    # 위의 dataset에 담긴 학습데이터를 batch만큼 나누는데 데이터양이 적기에 전체 크기(6)만큼 한번에 학습.
    # features가 x 데이터고 labels가 y 데이터다.
    for features, labels  in iter(dataset.batch(len(x_train))):
        hypothesis = logistic_regression(features)
        grads = grad(features, labels)
        # 경사 하강 수행
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
        if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(hypothesis,labels)))

Iter: 0, Loss: 0.6931
Iter: 100, Loss: 0.5781
Iter: 200, Loss: 0.5352
Iter: 300, Loss: 0.5056
Iter: 400, Loss: 0.4840
Iter: 500, Loss: 0.4673
Iter: 600, Loss: 0.4537
Iter: 700, Loss: 0.4421
Iter: 800, Loss: 0.4320
Iter: 900, Loss: 0.4229
Iter: 1000, Loss: 0.4145


In [5]:
# 테스트 데이터를 모델에 적용.
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Test Result = {}".format(tf.cast(logistic_regression(x_test) > 0.5, dtype=tf.int32)))  
print("Testset Accuracy: {:.4f}".format(test_acc))

Test Result = [[1]]
Testset Accuracy: 1.0000
